In [75]:
import numpy as np
import pandas as pd
import pickle
import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.metrics import classification_report

Load dataset and get binary and multiclass y columns.

In [76]:
binary = pd.read_csv("binary.csv")
binary.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,6,119360531,136,213,41528.0,155400.0,2955.0,0.0,305.352941,610.445587,...,1.522822e+06,2403023.0,249428.0,58229360.5,565451.372605,58629195.0,57829526.0,0.0,0,BENIGN
1,6,118838133,40,43,4216.0,16444.0,696.0,0.0,105.400000,188.610683,...,8.040554e+05,1152339.0,15233.0,58827757.0,223646.561181,58985899.0,58669615.0,0.0,0,BENIGN
2,6,108,1,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,BENIGN
3,17,21925,2,2,62.0,94.0,31.0,31.0,31.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,BENIGN
4,6,141,1,2,6.0,12.0,6.0,6.0,6.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,BENIGN


In [77]:
y = binary[" Label"].copy()
X = binary.drop([" Label"], axis=1)
y_bin = [0 if x=="BENIGN" else 1 for x in y.values]

print("The text labels from the encoding will be passed to classification report so we can interpret our results more easily.\n")
binary_labels = ["BENIGN", "ATTACK"]

print("Binary Label Encodings (in order of digits 0 -> 1): ")
print(binary_labels)

y_multi = y

# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()
y_multi = le.fit_transform(y_multi.values)

# Offset encoding so it corresponds to our original encoding (no benign column, which is now -1)
y_multi = [x-1 for x in y_multi]

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(y_multi))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)

The text labels from the encoding will be passed to classification report so we can interpret our results more easily.

Binary Label Encodings (in order of digits 0 -> 1): 
['BENIGN', 'ATTACK']

Multiclass Label Encodings (in order of digits 0 -> n): 
['BENIGN', 'DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load models

In [123]:
# Load a binary model
stage_1 = pickle.load(open("binary_model_test.pickle", 'rb'))

# Load a multiclass model
stage_2 = keras.models.load_model('LSTM_big.h5')

Stage 1 Prediction (Binary)

In [79]:
y_pred = stage_1.predict(X)

In [129]:
print(classification_report(y_bin, y_pred, digits=6, target_names=binary_labels))

              precision    recall  f1-score   support

      BENIGN   0.999322  0.998662  0.998992     54571
      ATTACK   0.998663  0.999322  0.998992     54571

    accuracy                       0.998992    109142
   macro avg   0.998992  0.998992  0.998992    109142
weighted avg   0.998992  0.998992  0.998992    109142



Housekeeping to prepare the dataset for 2nd stage.

In [106]:
# Create new dataframe out of binary predictions and multiclass labels that can be used to compare them later.
X_agg = X.copy()
X_agg["binary_pred"] = y_pred.copy()
X_agg[" Labels"] = y_multi.copy()

# Slice out the portions that we've identified as benign and replace their label with that of the multiclass benign label for comparison later.
X_rem = X_agg[X_agg["binary_pred"] == 0]
X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)

# Slice out the portions we've labeled as attack for 2nd stage classification.
X_mult = X_agg[X_agg["binary_pred"] == 1]
X_mult = X_mult.drop(["binary_pred"], axis = 1)
y_mult = X_mult[" Labels"].copy()
X_mult = X_mult.drop([" Labels"], axis = 1)

C:\Users\icarus\AppData\Local\Temp\ipykernel_38016\3257394839.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)


In [124]:
# Some reshaping is needed for the LSTM model to work
scal = StandardScaler()
X_tr = scal.fit_transform(X_mult)
X_tr = np.reshape(X_tr, (X_mult.shape[0], 1, X_mult.shape[1]))

y_pred2 = stage_2.predict(X_tr)
y_pred2 = [np.argmax(x) for x in y_pred2]

1707/1707 [==============================] - 4s 2ms/step


In [128]:
print(classification_report(y_mult, y_pred2, digits=6, target_names=multiclass_labels))

              precision    recall  f1-score   support

      BENIGN   0.000000  0.000000  0.000000        73
         DNS   0.443235  0.665792  0.532183      4955
        LDAP   0.168761  0.018952  0.034076      4960
       MSSQL   0.509288  0.929566  0.658047      4955
         NTP   0.973553  0.824788  0.893017      4954
     NetBIOS   0.953339  0.984476  0.968657      4960
        SNMP   0.592165  0.816569  0.686494      4961
        SSDP   0.403395  0.349798  0.374690      4960
         Syn   0.596165  0.998384  0.746546      4952
        TFTP   0.987369  0.551724  0.707891      4959
         UDP   0.473809  0.485178  0.479426      4959
     UDP-lag   0.972917  0.188344  0.315594      4959

    accuracy                       0.618492     54607
   macro avg   0.589500  0.567798  0.533052     54607
weighted avg   0.642222  0.618492  0.580679     54607



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Final Calculation of Results

In [131]:
y_pred_final = list(X_rem["binary_pred"].values.copy())
y_pred_final.extend(y_pred2)

y_true_final = list(X_rem[" Labels"].values.copy())
y_true_final.extend(y_mult.values)

print(classification_report(y_true_final, y_pred_final, digits=6, target_names=multiclass_labels))

              precision    recall  f1-score   support

      BENIGN   0.999322  0.998662  0.998992     54571
         DNS   0.443235  0.664987  0.531925      4961
        LDAP   0.168761  0.018948  0.034070      4961
       MSSQL   0.509288  0.928442  0.657765      4961
         NTP   0.973553  0.823624  0.892335      4961
     NetBIOS   0.953339  0.984277  0.968561      4961
        SNMP   0.592165  0.816569  0.686494      4961
        SSDP   0.403395  0.349728  0.374649      4961
         Syn   0.596165  0.996573  0.746039      4961
        TFTP   0.987369  0.551502  0.707708      4961
         UDP   0.473809  0.484983  0.479331      4961
     UDP-lag   0.972917  0.188268  0.315487      4961

    accuracy                       0.808781    109142
   macro avg   0.672776  0.650547  0.616113    109142
weighted avg   0.821206  0.808781  0.790149    109142

